In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

import time
import pandas as pd


## Getting started and connected

In [3]:
PATH = "C:\Program Files (x86)\web_driver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get('https://www.petco.com/shop/en/petcostore/category/dog/dog-food/wet-dog-food')

## Closes pop-up (remove if not necessary)

In [4]:
popup_close = WebDriverWait(driver, 13).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[14]/div[3]/div/div/div/div[2]/form/div[4]/div[3]/button"))
    )
popup_close.click()

## Extracts Links and Paginates

In [5]:
prodLinks=[]
page_count = 1
while True:
    page_count += 1
    
    # extract links
    food1 = driver.find_elements_by_class_name("product-name")
    for tags in food1:
        food2 = tags.find_elements_by_tag_name("a")
        for link in food2:
            prodLinks.append(link.get_attribute("href"))
    # go through each page
    try:
        time.sleep(7)
        driver.find_element_by_link_text(str(page_count)).click()
        # another is added to keep the page 
        time.sleep(7)
        driver.find_element_by_link_text(str(page_count)).click()
    # stop if no more pages available
    except NoSuchElementException:
        break


In [6]:
# total links scraped
len(prodLinks)

708

In [7]:
# preview of some of the links in a list
prodLinks[:6]

['https://www.petco.com/shop/en/petcostore/product/wet-noses-wag-soup-turkey-and-pumpkin-wet-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/merrick-grain-free-wingaling-wet-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/merrick-slow-cooked-bbq-texas-style-beef-recipe-wet-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/merrick-grain-free-cowboy-cookout-wet-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/hills-science-diet-adult-perfect-weight-hearty-vegetable-and-chicken-stew-canned-dog-food',
 'https://www.petco.com/shop/en/petcostore/product/merrick-slow-cooked-bbq-memphis-style-chicken-recipe-wet-dog-food']

## Extracts info from each link

In [ ]:
from tqdm import tqdm
data = []
# this for loop goes through only 5 out of 708 links
for i in tqdm(prodLinks[:5]):
    driver.get(i)
    
    name = driver.find_element_by_xpath('//*[@id="__next"]/main/div/div[1]/h1').text
    price = driver.find_element_by_xpath('//*[@id="__next"]/main/div/div[3]/form/div[2]/div[1]').text

    try:
        show_more = WebDriverWait(driver, 17).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/div[7]/ul/li[1]/div[2]/button'))
        )
        show_more.click()
        
        
        ids = driver.find_element_by_xpath('//*[@id="__next"]/main/div/div[7]/ul/li[1]/div[2]/div/div[1]/div/div/div[2]/ul/li[1]/p').text
        brand = driver.find_element_by_xpath('//*[@id="__next"]/main/div/div[7]/ul/li[1]/div[2]/div/div/div/div/div[2]/ul/li[4]/p').text
        primary_flavor = driver.find_element_by_xpath('//*[@id="__next"]/main/div/div[7]/ul/li[1]/div[2]/div/div/div/div/div[2]/ul/li[3]/p').text

    except NoSuchElementException:
        pass
    
    #collect the data
    wet_food = {'ID': ids,
                'brand': brand,
                'primary_flavor': primary_flavor,
                'name': name,
                'price': price}
    data.append(wet_food)

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:24<00:16,  8.14s/it]

In [ ]:
data

In [ ]:
petco_df = pd.DataFrame(data)
petco_df

## Quick Data Cleaning

In [ ]:
petco_df['price'] = petco_df['price'].apply(lambda x: x.split('\nreg'))
petco_df['new_price'] = petco_df['price'].apply(lambda x: x[0])
petco_df['reg_price'] = petco_df['price'].apply(lambda x: x[-1])
petco_df.drop('price', axis=1, inplace=True)

In [ ]:
petco_df